# Debugging

## Set up

In [75]:
suppressMessages({
    require(RColorBrewer, quietly=TRUE)
    require(data.table, quietly=TRUE)
    require(ggplot2, quietly=TRUE)
    require(magrittr, quietly=TRUE)
})

## Transaction

In [76]:
txGenerated <- fread("tx-generated.csv", stringsAsFactors=TRUE)
txGenerated

time_s,tx,node
<dbl>,<int>,<fct>
869.21,80921,node-413


In [79]:
txReceived <- fread("tx-received.csv", stringsAsFactors=TRUE)
txReceived %>% summary

     time_s            tx              node    
 Min.   :869.2   Min.   :80921   node-0  :  1  
 1st Qu.:869.3   1st Qu.:80921   node-1  :  1  
 Median :869.3   Median :80921   node-10 :  1  
 Mean   :869.4   Mean   :80921   node-100:  1  
 3rd Qu.:869.4   3rd Qu.:80921   node-101:  1  
 Max.   :870.2   Max.   :80921   node-102:  1  
                                 (Other) :743  

In [83]:
txReceived[, .(`Any non-receipts?`=.N!=749, `Last receipt [s]`=max(`time_s`), `Created [s]`=txGenerated[, `time_s`], `Elapsed [s]`=max(`time_s`)-txGenerated[, `time_s`])]

Any non-receipts?,Last receipt [s],Created [s],Elapsed [s]
<lgl>,<dbl>,<dbl>,<dbl>
FALSE,870.2488,869.21,1.0388


## RBs

In [32]:
rbGenerated <- fread("rb-generated.csv", stringsAsFactors=TRUE)
rbGenerated %>% summary

     time_s                    rb               parent               eb    
 Min.   :  20.07   100-node-432 : 1   1116-node-590: 3   1089-node-587: 2  
 1st Qu.: 518.08   1007-node-564: 1   1089-node-587: 2   1023-node-151: 1  
 Median : 855.08   1023-node-151: 1   100-node-432 : 1   105-node-598 : 1  
 Mean   : 808.93   105-node-598 : 1   1007-node-564: 1   1052-node-214: 1  
 3rd Qu.:1161.08   1052-node-214: 1   1023-node-151: 1   1127-node-567: 1  
 Max.   :1468.08   1089-node-587: 1   105-node-598 : 1   (Other)      :20  
                   (Other)      :71   (Other)      :68   NA's         :51  
   has_tx       
 Mode :logical  
 FALSE:77       
                
                
                
                
                

In [85]:
rbGenerated[`time_s` >= 850 & `time_s` <= 1100]

time_s,rb,parent,eb,has_tx
<dbl>,<fct>,<fct>,<fct>,<lgl>
855.075,855-node-429,839-node-148,NA,FALSE
864.075,864-node-101,855-node-429,NA,FALSE
920.075,920-node-423,864-node-101,864-node-101,FALSE
923.075,923-node-101,920-node-423,NA,FALSE
940.075,940-node-573,923-node-101,NA,FALSE
943.075,943-node-91,940-node-573,NA,FALSE
952.075,952-node-209,943-node-91,NA,FALSE
968.075,968-node-586,952-node-209,NA,FALSE
984.075,984-node-139,968-node-586,NA,FALSE


In [91]:
rbReceived <- fread("rb-received.csv", stringsAsFactors=TRUE)
rbReceived %>% summary

     time_s                    rb           recipient    
 Min.   :  20.08   1052-node-214: 1240   node-450:  152  
 1st Qu.: 552.58   1384-node-570: 1235   node-454:  152  
 Median : 923.79   1441-node-529: 1233   node-746:  152  
 Mean   : 839.26   1266-node-187: 1231   node-312:  151  
 3rd Qu.:1201.32   26-node-95   : 1226   node-470:  151  
 Max.   :1470.56   1195-node-530: 1224   node-488:  151  
                   (Other)      :61254   (Other) :67734  

In [104]:
rbGenerated[, .(`Created [s]`=`time_s`, `rb`)][
    rbReceived[, .(`Received [s]`=min(`time_s`)), .(`rb`, `recipient`)][, .(`Any non-receipts?`=.N<749, `Last receipt [s]`=max(`Received [s]`)), .(`rb`)],
    on="rb"
][, .(`rb`, `Created [s]`, `Any non-receipts?`, `Last receipt [s]`, `Elapsed [s]`=`Last receipt [s]`-`Created [s]`)] %>% summary

             rb      Created [s]      Any non-receipts? Last receipt [s]
 100-node-432 : 1   Min.   :  20.07   Mode :logical     Min.   :  21.8  
 1007-node-564: 1   1st Qu.: 518.08   FALSE:74          1st Qu.: 520.0  
 1023-node-151: 1   Median : 855.08   TRUE :3           Median : 857.0  
 105-node-598 : 1   Mean   : 808.93                     Mean   : 810.8  
 1052-node-214: 1   3rd Qu.:1161.08                     3rd Qu.:1162.9  
 1089-node-587: 1   Max.   :1468.08                     Max.   :1470.0  
 (Other)      :71                                                       
  Elapsed [s]   
 Min.   :1.628  
 1st Qu.:1.806  
 Median :1.887  
 Mean   :1.911  
 3rd Qu.:1.935  
 Max.   :2.743  
                

In [110]:
rbReceived[, `recipient`=unique(`recipient`)]

[1] 750

In [115]:
rbReceived[, .(`recipient`=unique(`recipient`))][rbReceived, on="recipient"] %>% summary

    recipient         time_s                    rb       
 node-450:  152   Min.   :  20.08   1052-node-214: 1240  
 node-454:  152   1st Qu.: 552.58   1384-node-570: 1235  
 node-746:  152   Median : 923.79   1441-node-529: 1233  
 node-312:  151   Mean   : 839.26   1266-node-187: 1231  
 node-470:  151   3rd Qu.:1201.32   26-node-95   : 1226  
 node-488:  151   Max.   :1470.56   1195-node-530: 1224  
 (Other) :67734                     (Other)      :61254  

The transaction was not directly included in an RB.

In [33]:
rbGenerated[`has_tx` == TRUE]

time_s,rb,parent,eb,has_tx
<dbl>,<fct>,<fct>,<fct>,<lgl>


In [22]:
ebGenerated <- fread("eb-generated.csv", stringsAsFactors=TRUE)
ebGenerated %>% summary

     time_s             node               eb   
 Min.   : 920.1   node-101:1   1007-node-564:1  
 1st Qu.: 940.1   node-139:1   920-node-423 :1  
 Median : 952.1   node-209:1   923-node-101 :1  
 Mean   : 958.4   node-423:1   940-node-573 :1  
 3rd Qu.: 984.1   node-486:1   943-node-91  :1  
 Max.   :1007.1   node-564:1   952-node-209 :1  
                  (Other) :3   (Other)      :3  

In [23]:
ebReceived <- fread("eb-received.csv", stringsAsFactors=TRUE)
ebReceived %>% summary

     time_s                   eb          recipient   
 Min.   : 920.3   1007-node-564: 749   node-0  :   9  
 1st Qu.: 940.9   920-node-423 : 749   node-1  :   9  
 Median : 953.0   923-node-101 : 749   node-10 :   9  
 Mean   : 959.5   940-node-573 : 749   node-100:   9  
 3rd Qu.: 985.3   943-node-91  : 749   node-102:   9  
 Max.   :1010.0   952-node-209 : 749   node-103:   9  
                  (Other)      :2247   (Other) :6687  

None of the EBs including the transaction were endorsed.

In [36]:
ebGenerated[rbGenerated, on="eb", nomatch=0]

time_s,node,eb,i.time_s,rb,parent,has_tx
<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<lgl>


In [38]:
ebGenerated

time_s,node,eb
<dbl>,<fct>,<fct>
920.075,node-423,920-node-423
923.075,node-101,923-node-101
940.075,node-573,940-node-573
943.075,node-91,943-node-91
952.075,node-209,952-node-209
968.075,node-586,968-node-586
984.075,node-139,984-node-139
988.075,node-486,988-node-486
1007.075,node-564,1007-node-564


In [72]:
ebGenerated[, .(`eb`, `Created [s]`=`time_s`)][
    ebReceived[, .(`Any non-receipts?`=.N!=749, `Last receipt [s]`=max(`time_s`)), .(`eb`)],
    on="eb"
][, .(`eb`, `Created [s]`, `Any non-receipts?`, `Last receipt [s]`, `Elapsed [s]`=`Last receipt [s]`-`Created [s]`)]

eb,Created [s],Any non-receipts?,Last receipt [s],Elapsed [s]
<fct>,<dbl>,<lgl>,<dbl>,<dbl>
920-node-423,920.075,FALSE,922.0910,2.0160
923-node-101,923.075,FALSE,925.1240,2.0490
940-node-573,940.075,FALSE,942.2498,2.1748
943-node-91,943.075,FALSE,945.2436,2.1686
952-node-209,952.075,FALSE,954.3091,2.2341
968-node-586,968.075,FALSE,970.3027,2.2277
984-node-139,984.075,FALSE,986.2630,2.1880
988-node-486,988.075,FALSE,990.3145,2.2395
1007-node-564,1007.075,FALSE,1010.0482,2.9732


In [24]:
vtGenerated <- fread("vt-generated.csv", stringsAsFactors=TRUE)
vtGenerated %>% summary

     time_s             node                 vote                  eb     
 Min.   : 920.1   node-1  :   9   1007-node-1  :   1   968-node-586 :200  
 1st Qu.: 943.0   node-100:   9   1007-node-100:   1   943-node-91  :192  
 Median : 955.0   node-101:   9   1007-node-101:   1   952-node-209 :189  
 Mean   : 961.4   node-12 :   9   1007-node-12 :   1   1007-node-564:188  
 3rd Qu.: 987.0   node-132:   9   1007-node-13 :   1   920-node-423 :188  
 Max.   :1011.7   node-135:   9   1007-node-131:   1   988-node-486 :188  
                  (Other) :1649   (Other)      :1697   (Other)      :558  
     weight     
 Min.   :1.000  
 1st Qu.:1.000  
 Median :2.000  
 Mean   :2.554  
 3rd Qu.:3.000  
 Max.   :8.000  
                

In [25]:
vtReceived <- fread("vt-received.csv", stringsAsFactors=TRUE)
vtReceived %>% summary

     time_s                  vote            recipient      
 Min.   : 920.1   1007-node-1  :    749   node-10 :   1703  
 1st Qu.: 943.1   1007-node-100:    749   node-102:   1703  
 Median : 955.1   1007-node-101:    749   node-103:   1703  
 Mean   : 961.7   1007-node-12 :    749   node-104:   1703  
 3rd Qu.: 987.3   1007-node-13 :    749   node-105:   1703  
 Max.   :1013.6   1007-node-131:    749   node-106:   1703  
                  (Other)      :1271053   (Other) :1265329  

In [74]:
ebGenerated[, .(`eb`, `Created [s]`=`time_s`)][
    vtGenerated[, .(`eb`, `vote`, `weight`)][
        vtReceived[, .(`Any non-receipts?`=.N!=749, `Last receipt [s]`=max(`time_s`)), .(`vote`)],
        on="vote"
    ][, .(`weight`=sum(`weight`), `Any non-receipts?`=any(`Any non-receipts?`), `Last receipt [s]`=max(`Last receipt [s]`)), .(`eb`)],
    on="eb"
][, .(`eb`, `Created [s]`, `weight`, `Any non-receipts?`, `Last receipt [s]`, `Elapsed [s]`=`Last receipt [s]`-`Created [s]`)]

eb,Created [s],weight,Any non-receipts?,Last receipt [s],Elapsed [s]
<fct>,<dbl>,<int>,<lgl>,<dbl>,<dbl>
920-node-423,920.075,509,FALSE,925.0764,5.0014
923-node-101,923.075,454,FALSE,928.1464,5.0714
940-node-573,940.075,513,FALSE,945.5587,5.4837
943-node-91,943.075,497,FALSE,948.5800,5.5050
952-node-209,952.075,467,FALSE,957.8020,5.7270
968-node-586,968.075,473,FALSE,973.9178,5.8428
984-node-139,984.075,480,FALSE,989.8634,5.7884
988-node-486,988.075,497,FALSE,993.9080,5.8330
1007-node-564,1007.075,459,FALSE,1013.5602,6.4852


In [71]:
ebGenerated %>% head

time_s,node,eb
<dbl>,<fct>,<fct>
920.075,node-423,920-node-423
923.075,node-101,923-node-101
940.075,node-573,940-node-573
943.075,node-91,943-node-91
952.075,node-209,952-node-209
968.075,node-586,968-node-586
